## Directory Structure

The first step is to set up a rawdata, procdata, and results directory to keep track of the downloaded data, processing steps, and the final result. We'll also make a workflow directory to keep track of code, logs, configuration files, and other relevant files.

The organization will look like this:

```raw
├── README.md
├── procdata
├── rawdata
├── results
└── workflow
    ├── config
    ├── envs
    ├── logs
    └── scripts
```


#### Confirm working directory is correct...

In [1]:
import os
print("Working directory should end in workflow/scripts:", os.getcwd())

Working directory should end in workflow/scripts: /Users/katyscott/Documents/HNC Project/scripts/clinical_data_handling/workflow/scripts


In [3]:
!mkdir -p ../../rawdata/ ../../procdata/ ../../results/

# Download clinical data from TCIA for the HNSCC dataset

HNSCC is a publicly available dataset on TCIA. The imaging data is under a TCIA Restricted License, but the clinical data is available for download. 

There are two clinical spreadsheets provided as data from two institutions is included. The first is _Head-Neck-CT-Atlas_ and the second is _Radiomics outcome prediction in Oropharyngeal cancer_. We'll refer to these as ATLAS and OPC, respectively.

In [4]:
# Download ATLAS Clinical Data to the appropriate directory
# Won't download if the file already exists
!wget -nc -P ../../rawdata/HNSCC/clinical/atlas/ https://www.cancerimagingarchive.net/wp-content/uploads/HNSCC-MDA-Data_update_20240514.xlsx

File ‘../../rawdata/HNSCC/clinical/atlas/HNSCC-MDA-Data_update_20240514.xlsx’ already there; not retrieving.



In [8]:
# Download OPC Clinical Data to the appropriate directory
# Won't download if the file already exists
!wget -nc -P ../../rawdata/HNSCC/clinical/opc/ https://www.cancerimagingarchive.net/wp-content/uploads/Radiomics_Outcome_Prediction_in_OPC_ASRM_corrected.csv

--2024-09-30 17:18:39--  https://www.cancerimagingarchive.net/wp-content/uploads/Radiomics_Outcome_Prediction_in_OPC_ASRM_corrected.csv
Resolving www.cancerimagingarchive.net (www.cancerimagingarchive.net)... 144.30.169.13
Connecting to www.cancerimagingarchive.net (www.cancerimagingarchive.net)|144.30.169.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88002 (86K) [text/csv]
Saving to: ‘../../rawdata/HNSCC/clinical/opc/Radiomics_Outcome_Prediction_in_OPC_ASRM_corrected.csv’

Radiomics_Outcome_P 100%[===================>]  85.94K   431KB/s    in 0.2s    

2024-09-30 17:18:39 (431 KB/s) - ‘../../rawdata/HNSCC/clinical/opc/Radiomics_Outcome_Prediction_in_OPC_ASRM_corrected.csv’ saved [88002/88002]



: 

# Data Loading

In [2]:
import pandas as pd
import os
from helpers import *

In [13]:
atlas_clinical_data_path = os.path.join("../../rawdata/HNSCC/clinical/atlas/", os.listdir("../../rawdata/HNSCC/clinical/atlas/")[0])
atlas_clinical_dataframe = load_data_to_df(atlas_clinical_data_path)

In [14]:
opc_clinical_data_path = os.path.join("../../rawdata/HNSCC/clinical/opc/", os.listdir("../../rawdata/HNSCC/clinical/opc/")[0])
opc_clinical_dataframe =load_data_to_df(opc_clinical_data_path)

# Rename Clinical Columns to match

These two clinical datasets have different column numbers and names. We're going to rename the columns to match, based on the RADCURE clinical dataset (also on TCIA).

In [15]:
updated_col_names_atlas = {"Alive or Dead": "Status"}

updated_atlas_clinical_dataframe = atlas_clinical_dataframe.rename(columns=updated_col_names_atlas)

In [16]:
updated_col_names_opc = {"Gender": "Sex",
                         "Age at Diag": "Age",
                         "T-category": "T",
                         "N-category": "N",
                         "AJCC Stage (7th edition)": "Stage",
                         "Total prescribed Radiation treatment dose": "RT Total Dose (Gy)",
                         "Radiation treatment_number of fractions": "Number of Fractions",
                         "Radiation treatment_dose per fraction": "Dose/Fraction (Gy/fx)",
                         "Vital status": "Status",
                         "Overall survival_duration of Merged updated ASRM V2": "Survival (days)",
                         }

updated_opc_clinical_dataframe = opc_clinical_dataframe.rename(columns=updated_col_names_opc)

### Rename the patient ID column

In [17]:
atlas_patient_identifier = getPatientIdentifierLabel(updated_atlas_clinical_dataframe)
updated_atlas_clinical_dataframe.rename(columns={atlas_patient_identifier: "patient_ID"}, inplace=True)

In [18]:
opc_patient_identifier = getPatientIdentifierLabel(updated_opc_clinical_dataframe)
updated_opc_clinical_dataframe.rename(columns={opc_patient_identifier: "patient_ID"}, inplace=True)

Multiple patient identifier labels found. Using the first one.


### Make a Survival (years) column for the datasets

In [22]:
# Get survival in years for both datasets

atlas_survival = updated_atlas_clinical_dataframe[f"Survival  (months)"]
atlas_survival_years = atlas_survival / 12
updated_atlas_clinical_dataframe[f"Survival (years)"] = atlas_survival_years

opr_survival = updated_opc_clinical_dataframe[f"Survival (days)"]
opr_survival_years = opr_survival / 365
updated_opc_clinical_dataframe[f"Survival (years)"] = opr_survival_years

In [16]:
# OPTIONAL: Save the updated versions of the clinical data
# updated_atlas_clinical_dataframe.to_csv("../../procdata/HSNCC_atlas_updated_colnames_clinical_data.csv")
# updated_opc_clinical_dataframe.to_csv("../../procdata/HSNCC_opc_updated_colnames_clinical_data.csv")

# Merge Dataframes

Now that the datasets have more matching columns, we can merge them into a single dataframe. The ATLAS dataset has many columns not present in OPC and vice versa, so these will be marked as `NaN` in the merged dataframe.

## Find common columns
Find the columns that are in both datasets. 

For the patients that are unique to one or the other, we'll use this to merge the datasets.

**For the patients in both datasets, we will use the data from the ATLAS dataset.**

In [23]:
# Get common columns in the two clinical datasets - this will be the columns we'll merge on
common_cols = list(set(updated_atlas_clinical_dataframe.columns) & set(updated_opc_clinical_dataframe.columns))

In [24]:
# Get overlapping patient IDs
atlas_patients = updated_atlas_clinical_dataframe['patient_ID']
opc_patients = updated_opc_clinical_dataframe['patient_ID']

overlapping_patients = list(set(atlas_patients) & set(opc_patients))

### Merge the unique patients between the two clinical datasets.

Each patient only has data in one of the clinical datasets, so we can join on all of the common columns to prevent duplicates of those.

The output will include a column at the end named "_merge" that indicated which dataset the patient came from.

In [25]:
unique_atlas_clinical_dataframe = updated_atlas_clinical_dataframe[~updated_atlas_clinical_dataframe['patient_ID'].isin(overlapping_patients)]
unique_opc_clinical_dataframe = updated_opc_clinical_dataframe[~updated_opc_clinical_dataframe['patient_ID'].isin(overlapping_patients)]

merged_unique_clinical_dataframe = pd.merge(unique_atlas_clinical_dataframe, unique_opc_clinical_dataframe, 
                                            how='outer', 
                                            on=common_cols, 
                                            suffixes=('_ATLAS', '_OPC'),
                                            indicator=True)

### Find patients that overlap between the two clinical datasets

Each of these patients exist in both datasets. Some of the clinical data is the same, but some is different. Here, we find all the common columns between the two datasets and **only use the data from the ATLAS dataset**. 

In [26]:
# Filter out the overlapping patients
overlap_atlas_clinical_dataframe = updated_atlas_clinical_dataframe[updated_atlas_clinical_dataframe['patient_ID'].isin(overlapping_patients)]
overlap_opc_clinical_dataframe = updated_opc_clinical_dataframe[updated_opc_clinical_dataframe['patient_ID'].isin(overlapping_patients)]

# Drop the common columns from the OPC dataframe except for the patient ID
non_id_common_columns = common_cols
non_id_common_columns.remove('patient_ID')

overlap_opc_clinical_dataframe = overlap_opc_clinical_dataframe.drop(columns=non_id_common_columns)

# Merge the two dataframes
merged_overlap_clinical_dataframe = pd.merge(overlap_atlas_clinical_dataframe, overlap_opc_clinical_dataframe, 
                                             how='left', 
                                             on=['patient_ID'], 
                                             suffixes=('_ATLAS', '_OPC'), 
                                             indicator=True)

## Confirm that the number of columns is the same in the two merged datasets

In [28]:
print("The second value for each dataset should be 94:")
print("Unique patient dataframe size:", merged_unique_clinical_dataframe.shape)
print("Overlap patient dataframe size:", merged_overlap_clinical_dataframe.shape)

The second value for each dataset should be 94:
Unique patient dataframe size: (547, 94)
Overlap patient dataframe size: (80, 94)


## Concatenate and sort the resulting dataframes

In [29]:
combined_clinical_dataframe = pd.concat([merged_overlap_clinical_dataframe, merged_unique_clinical_dataframe], axis=0)
combined_clinical_dataframe.sort_values(by=['patient_ID'], inplace=True)

# Save the merged dataframe to a csv file

In [30]:
from datetime import datetime
todays_date = datetime.today().strftime('%Y%m%d')
combined_clinical_dataframe.to_csv("../../results/HNSCC_combined_atlas_and_opc_clinical_data_" + todays_date +".csv", index=False)